In [28]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

In [29]:
# Create data base file
disk_engine = create_engine('sqlite:///PerformanceData.db')


###### Read the file 
# Read the CSV to the DB by chunksize

start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

years = ['2011-12', '2013', '2014-H1', '2014-H2', '2015-H1', '2015-H2', '2016-H1', '2016-H2', '2017-H1']

for y in years:
    for df in pd.read_csv('Data\CompanyX ' + y + ' Loan Vintage Transactions 2017-06-30.csv', chunksize=chunksize, iterator=True, encoding="ISO-8859-1"):
        df.index += index_start
        j+=1
        print ('Loading file...',j*chunksize)
        df.to_sql('LoanData', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1
    

Loading file... 20000
Loading file... 40000
Loading file... 60000
Loading file... 80000
Loading file... 100000
Loading file... 120000
Loading file... 140000
Loading file... 160000
Loading file... 180000
Loading file... 200000
Loading file... 220000
Loading file... 240000
Loading file... 260000
Loading file... 280000
Loading file... 300000
Loading file... 320000
Loading file... 340000
Loading file... 360000
Loading file... 380000
Loading file... 400000
Loading file... 420000
Loading file... 440000
Loading file... 460000
Loading file... 480000
Loading file... 500000
Loading file... 520000
Loading file... 540000
Loading file... 560000
Loading file... 580000
Loading file... 600000
Loading file... 620000
Loading file... 640000
Loading file... 660000
Loading file... 680000
Loading file... 700000
Loading file... 720000
Loading file... 740000
Loading file... 760000
Loading file... 780000
Loading file... 800000
Loading file... 820000
Loading file... 840000
Loading file... 860000
Loading file...

In [32]:
# Read the name of the tables in the DB
disk_engine = create_engine('sqlite:///PerformanceData.db')

#Read the name of tables from the DB
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name",disk_engine)

pd.read_sql_query('SELECT MAX(LoanVintage) FROM LoanData', disk_engine)


#Delete table
#pd.read_sql_query('DROP TABLE LoanData;',disk_engine)

,MAX(LoanVintage)
0,9/30/2016


In [21]:
##### SQL command
qry = """
        SELECT  
            LoanVintage, 
            substr(TransactionDate,1,4) || substr(TransactionDate,5,2) As TransactionVintage,
            ProductType,
            Sum(PrincipalChargeOff) As ChargeOffAmount,
            Round(FICOQual/20-0.5)*20 AS FICO,
            Round(KSFScore/20-0.5)*20 AS KS
        FROM
            LoanData
        WHERE 
            PrincipalChargeOff<0
        GROUP BY
            LoanVintage,
            substr(TransactionDate,1,4) || substr(TransactionDate,5,2),
            ProductType,
            Round(FICOQual/20-0.5)*20,
            Round(KSFScore/20-0.5)*20;
        """

df = pd.read_sql_query(qry, disk_engine)

# Add df to a table called new
#df.to_sql('new', disk_engine, if_exists='append')
# Export df
df.to_csv('Output\\ChargeOffData.csv', sep=';', encoding='utf-8')

In [25]:
##### SQL command
qry = """
        SELECT
            A.LoanVintage, 
            A.ProductType,
            Sum(A.OriginalPurchaseAmount) As OriginationAmount,
            A.FICO,
            A.KS
        FROM
            (
            SELECT  
                LoanIdentifier,
                LoanVintage, 
                ProductType,
                OriginalPurchaseAmount,
                Round(FICOQual/20-0.5)*20 AS FICO,
                Round(KSFScore/20-0.5)*20 AS KS
            FROM
                LoanData
            GROUP BY
                LoanIdentifier,
                LoanVintage,
                ProductType,
                OriginalPurchaseAmount,
                Round(FICOQual/20-0.5)*20,
                Round(KSFScore/20-0.5)*20
            ) As A
        GROUP BY
            A.LoanVintage,
            A.ProductType,
            A.FICO,
            A.KS
        HAVING 
            Sum(A.OriginalPurchaseAmount)>0;
        """

df = pd.read_sql_query(qry, disk_engine)
#display(df)

# Add df to a table called new
#df.to_sql('new', disk_engine, if_exists='append')
# Export df
df.to_csv('Output\\OriginationAmount.csv', sep=';', encoding='utf-8')

In [27]:
df = pd.read_sql_query('SELECT MAX(LoanVintage) FROM LoanData', disk_engine)
display(df)

,MAX(LoanVintage)
0,9/30/2016
